In [ ]:
!pip install pdfplumber
!pip install deep_translator
!pip install fpdf
!pip install telebot

SyntaxError: invalid decimal literal (<ipython-input-3-6ba254ee07e1>, line 4)

In [ ]:
from deep_translator import GoogleTranslator
import pdfplumber
import telebot
from nltk.tokenize import sent_tokenize
from fpdf import FPDF
import nltk
nltk.download('punkt')
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def extract(page):
    """Extract PDF text and Delete in-paragraph line breaks."""
    # Get text
    extracted = page.extract_text()
    # Delete in-paragraph line breaks
    extracted = extracted.replace(".\n", "**/m" # keep par breaks
                        ).replace(". \n", "**/m" # keep par breaks
                        ).replace("\n", "" # delete in-par breaks
                        ).replace("**/m", ".\n\n") # restore par break
    return extracted

In [ ]:
def translate_extracted(Extracted):
    """Wrapper for Google Translate with upload workaround."""
    # Set-up and wrap translation client
    translate = GoogleTranslator(source='auto', target='en').translate
    # Split input text into a list of sentences
    sentences = sent_tokenize(Extracted)
    # Initialize containers
    translated_text = ''
    source_text_chunk = ''
    # collect chunks of sentences, translate individually
    for sentence in sentences:
        # if chunk + current sentence < limit, add the sentence
        if ((len(sentence.encode('utf-8')) +  len(source_text_chunk.encode('utf-8')) < 5000)):
            source_text_chunk += ' ' + sentence
        # else translate chunk and start new one with current sentence
        else:
            translated_text += ' ' + translate(source_text_chunk)
        # if current sentence smaller than 5000 chars, start new chunk
        if (len(sentence.encode('utf-8')) < 5000):
            source_text_chunk = sentence
        # else, replace sentence with notification message
        else:
            message = "<<Omitted Word longer than 5000bytes>>"
            translated_text += ' ' + translate(message)
            # Re-set text container to empty
            source_text_chunk = ''
    # Translate the final chunk of input text, if there is any valid text left to translate
    if translate(source_text_chunk) != None:
        translated_text += ' ' + translate(source_text_chunk)
    return translated_text

In [ ]:
# Initialize the bot with the bot's token
bot = telebot.TeleBot("Token")

def translate_text(text):
    translator = GoogleTranslator(source='auto', target='ar')
    translated_text = translator.translate(text)
    return translated_text

# Define the extraction function
def extract(page):
    """Extract PDF text and Delete in-paragraph line breaks."""
    # Get text
    extracted = page.extract_text()
    # Delete in-paragraph line breaks
    extracted = extracted.replace(".\n", "**/m" # keep par breaks
                        ).replace(". \n", "**/m" # keep par breaks
                        ).replace("\n", "" # delete in-par breaks
                        ).replace("**/m", ".\n\n") # restore par break
    return extracted

# Handler for the /start command
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome to the PDF Translator bot! Send me a PDF file.")

# Handler for messages containing PDF files
@bot.message_handler(content_types=['document'])
def handle_pdf(message):
    #sent PDF
    if message.document.mime_type == 'application/pdf':
        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        #save sent PDF
        with open('input.pdf', 'wb') as new_file:
            new_file.write(downloaded_file)

        # Create a choice menu
        keyboard = InlineKeyboardMarkup()
        keyboard.row_width = 2
        text_button = InlineKeyboardButton("Text", callback_data='text')
        pdf_button = InlineKeyboardButton("PDF", callback_data='pdf')
        keyboard.add(text_button, pdf_button)

        bot.send_message(message.chat.id, "Choose output format:", reply_markup=keyboard)
    else:
        bot.reply_to(message, "Please send a PDF file.")

# Handler for callback queries (i.e., when the user clicks on a button)
@bot.callback_query_handler(func=lambda call: True)
def callback_handler(call):
    if call.data == 'text':
        # Translate the PDF and send back the translated text
        translated_text = ""
        with pdfplumber.open('input.pdf') as pdf:
            for page in pdf.pages:
                # Extract Page
                extracted = extract(page)
                # Translate Page
                translated_text += translate_text(extracted) + "\n\n"

        with open('output.txt', 'w') as output_file:
            output_file.write(translated_text)
        # Send the text file
        bot.send_document(call.message.chat.id, open('output.txt', 'rb'))

    elif call.data == 'pdf':
        # Translate the PDF and send back a PDF file
        translated_text = ""
        with pdfplumber.open('input.pdf') as pdf:
            for page in pdf.pages:
                # Extract Page
                extracted = extract(page)
                # Translate Page
                translated_text += translate_text(extracted) + "\n\n"

        with open('output.pdf', 'w') as output_file:
            output_file.write(translated_text)

        bot.send_document(call.message.chat.id, open('output.pdf', 'rb'))

# Start the bot
bot.polling()